# This notebook is used to test and develop the integration of Kamodo into GEODYN


- We make a some critical modifications to GEODYN for the purpose of quick testing:
    1. We use a much lager `STEP` size to go through the runs faster.  
       `10.00 (s)  ->  60.00 (s)`
    


Before we run an unused set of TIEGCM data for the first time, we have to initialize the Data to work with the Kamodo Flythrough
 - kamodo automatically constructs some datatables for quick look ups of dates.
    

In [1]:
# import netCDF4

# ds = netCDF4.Dataset('/data/data_geodyn/atmos_models_data/tiegcm/2018/Lutz_Rastaetter_072319_IT_1/s001.nc')
# ds.variables.keys()

In [2]:
# rho = float(ds.variables['DEN'][12,12,12,4].data)

# n_n2 = float(ds.variables['N2N'][12,12,12,4].data)

# # den_N2 = results_dict['psi_He'][i] * results_dict['rho'][i]     # rho_He    [kg/cm^3]

# mbar_n2 = rho/n_n2
# mbar_n2

In [3]:
# # import sys
# # sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# # from SingleSatelliteFlythrough import SingleModelFlythrough
# # sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# # from SatelliteFlythrough import ModelFlythrough
# # import pandas as pd
# # from datetime import datetime, timezone, timedelta
# # import numpy as np

# # #### Kamodo static inputs:
# # model          = 'TIEGCM'
# # file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2018/AugustQuiet_JONAH_120419_IT_2/'
# # variable_list  = ['rho','psi_O2', 'psi_O', 'psi_N2', 'psi_He', 'T_n']
# # coord_type     = 'SPH'
# # coord_grid     = 'sph'
# # high_res       = 1.
# # verbose        = False  
# # csv_output     = '' 
# # plot_output    = ''


# # #### Extract the coordinates from each list to plug into Kamodo with vectorization

# # ## Gather inputs
# # sat_time       = datetime.timestamp(datetime.strptime('181202120000', '%y%m%d%H%M%S').replace(tzinfo=timezone.utc))
# # c1             = 1
# # c2             = 10
# # c3             = 500
    
# # ## Plug vectorized coordinates into Kamodo
# # results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
# #                     coord_type, coord_grid, high_res=20., verbose=False, 
# #                     csv_output='', plot_output='')

# import sys
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SingleSatelliteFlythrough import SingleModelFlythrough
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SatelliteFlythrough import ModelFlythrough
# import pandas as pd
# from datetime import datetime, timezone, timedelta
# import numpy as np

# #### Kamodo static inputs:
# model          = 'TIEGCM'
# file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2018/Lutz_Rastaetter_072319_IT_1/'
# variable_list  = ['rho','psi_O2', 'psi_O', 'psi_He', 'T_n']
# coord_type     = 'SPH'
# coord_grid     = 'sph'
# high_res       = 1.
# verbose        = False  
# csv_output     = '' 
# plot_output    = ''


# #### Extract the coordinates from each list to plug into Kamodo with vectorization

# ## Gather inputs
# sat_time       = [datetime.timestamp(datetime.strptime('181202120000', '%y%m%d%H%M%S').replace(tzinfo=timezone.utc))]
# c1             = [1]
# c2             = [10]
# c3             = [500]
    
# ## Plug vectorized coordinates into Kamodo
# results = ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, 
#                     coord_type, coord_grid, high_res=20., verbose=False, 
#                     csv_output='', plot_output='')

Make a csv to run the GEODYN dataset through

We now have a CSV.

What is next:
1. Run through the entire command line method for a benchmark
1. make a log file and if the time doesn't match up, grab the next closest time and write it to log file.\
1. put interpolate function in fortran

## Run the orbit cloud method thru pygeodyn

In [ ]:
%load_ext autoreload
%autoreload 2
import copy
import time
import sys  


sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn


### Identify which arcs you want to run:
arcs_files = [ 
#             '2018.335',  # 1
#             '2018.336',  # 1
            '2018.337',  # 2
#             '2018.338',  # 3
             ]

#------ A dictionary containing the run parameters ------  
run_params = {} 
run_params['arc']              =   arcs_files
run_params['satellite']        =  'icesat2'  
run_params['den_model']        =  'tiegcm_oc'  
run_params['SpecialRun_name']  =  'PCE_kamodo_interface'  
run_params['verbose']          =  False
run_params['geodyn_StepSize']  =  float(10.)


%load_ext autoreload
%autoreload 2

import pickle 
sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn
# import time
from time import perf_counter
t0=perf_counter()


for imodel,val_model in enumerate( ['tiegcm_oc']):
    run_params1 = copy.deepcopy(run_params)
    run_params1['den_model']  =  val_model  
    run_params1['action']     =  'run'

    ### Load the data into an object
    Obj_Geodyn = Pygeodyn(run_params1)
    Obj_Geodyn.RUN_GEODYN()
    del Obj_Geodyn
    print('time per run:', perf_counter()-t0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                      ......... RUNNING GEODYN
RUNNING THE ORBIT CLOUD METHOD
time at start: 2.2151507437229156e-06
import time: 2.2467472893185914
TEST1
**** 181202210109 -- 0 ****
**** 181202210119 -- 1 ****
**** 181202210129 -- 2 ****
**** 181202210139 -- 3 ****
**** 181202210149 -- 4 ****
**** 181202210059 -- 5 ****
**** 181202210049 -- 6 ****
**** 181202210039 -- 7 ****
**** 181202210029 -- 8 ****
**** 181202210019 -- 9 ****
**** 181202210149 -- 10 ****
**** 181202210139 -- 11 ****
**** 181202210129 -- 12 ****
**** 181202210119 -- 13 ****
**** 181202210059 -- 14 ****
**** 181202210049 -- 15 ****
**** 181202210039 -- 16 ****
**** 181202210029 -- 17 ****
**** 181202210019 -- 18 ****
**** 181202210149 -- 19 ****
**** 181202210139 -- 20 ****
**** 181202210129 -- 21 ****
**** 181202210119 -- 22 ****
**** 181202210059 -- 23 ****
**** 181202210049 -- 24 ****
**** 181202210039 -- 25 ****
**** 181202210

**** 181202213909 -- 269 ****
**** 181202213919 -- 270 ****
**** 181202213929 -- 271 ****
**** 181202213939 -- 272 ****
**** 181202213949 -- 273 ****
**** 181202213959 -- 274 ****
**** 181202214009 -- 275 ****
**** 181202214019 -- 276 ****
**** 181202214029 -- 277 ****
**** 181202214039 -- 278 ****
**** 181202214049 -- 279 ****
**** 181202214059 -- 280 ****
**** 181202214109 -- 281 ****
**** 181202214119 -- 282 ****
**** 181202214129 -- 283 ****
**** 181202214139 -- 284 ****
**** 181202214149 -- 285 ****
**** 181202214159 -- 286 ****
**** 181202214209 -- 287 ****
**** 181202214219 -- 288 ****
**** 181202214229 -- 289 ****
**** 181202214239 -- 290 ****
**** 181202214249 -- 291 ****
**** 181202214259 -- 292 ****
**** 181202214309 -- 293 ****
**** 181202214319 -- 294 ****
**** 181202214329 -- 295 ****
**** 181202214339 -- 296 ****
**** 181202214349 -- 297 ****
**** 181202214359 -- 298 ****
**** 181202214409 -- 299 ****
**** 181202214419 -- 300 ****
**** 181202214429 -- 301 ****
**** 18120

**** 181202222449 -- 543 ****
**** 181202222459 -- 544 ****
**** 181202222509 -- 545 ****
**** 181202222519 -- 546 ****
**** 181202222529 -- 547 ****
**** 181202222539 -- 548 ****
**** 181202222549 -- 549 ****
**** 181202222559 -- 550 ****
**** 181202222609 -- 551 ****
**** 181202222619 -- 552 ****
**** 181202222629 -- 553 ****
**** 181202222639 -- 554 ****
**** 181202222649 -- 555 ****
**** 181202222659 -- 556 ****
**** 181202222709 -- 557 ****
**** 181202222719 -- 558 ****
**** 181202222729 -- 559 ****
**** 181202222739 -- 560 ****
**** 181202222749 -- 561 ****
**** 181202222759 -- 562 ****
**** 181202222809 -- 563 ****
**** 181202222819 -- 564 ****
**** 181202222829 -- 565 ****
**** 181202222839 -- 566 ****
**** 181202222849 -- 567 ****
**** 181202222859 -- 568 ****
**** 181202222909 -- 569 ****
**** 181202222919 -- 570 ****
**** 181202222929 -- 571 ****
**** 181202222939 -- 572 ****
**** 181202222949 -- 573 ****
**** 181202222959 -- 574 ****
**** 181202223009 -- 575 ****
**** 18120

**** 181202231029 -- 817 ****
**** 181202231039 -- 818 ****
**** 181202231049 -- 819 ****
**** 181202231059 -- 820 ****
**** 181202231109 -- 821 ****
**** 181202231119 -- 822 ****
**** 181202231129 -- 823 ****
**** 181202231139 -- 824 ****
**** 181202231149 -- 825 ****
**** 181202231159 -- 826 ****
**** 181202231209 -- 827 ****
**** 181202231219 -- 828 ****
**** 181202231229 -- 829 ****
**** 181202231239 -- 830 ****
**** 181202231249 -- 831 ****
**** 181202231259 -- 832 ****
**** 181202231309 -- 833 ****
**** 181202231319 -- 834 ****
**** 181202231329 -- 835 ****
**** 181202231339 -- 836 ****
**** 181202231349 -- 837 ****
**** 181202231359 -- 838 ****
**** 181202231409 -- 839 ****
**** 181202231419 -- 840 ****
**** 181202231429 -- 841 ****
**** 181202231439 -- 842 ****
**** 181202231449 -- 843 ****
**** 181202231459 -- 844 ****
**** 181202231509 -- 845 ****
**** 181202231519 -- 846 ****
**** 181202231529 -- 847 ****
**** 181202231539 -- 848 ****
**** 181202231549 -- 849 ****
**** 18120

**** 181202235539 -- 1088 ****
**** 181202235549 -- 1089 ****
**** 181202235559 -- 1090 ****
**** 181202235609 -- 1091 ****
**** 181202235619 -- 1092 ****
**** 181202235629 -- 1093 ****
**** 181202235639 -- 1094 ****
**** 181202235649 -- 1095 ****
**** 181202235659 -- 1096 ****
**** 181202235709 -- 1097 ****
**** 181202235719 -- 1098 ****
**** 181202235729 -- 1099 ****
**** 181202235739 -- 1100 ****
**** 181202235749 -- 1101 ****
**** 181202235759 -- 1102 ****
**** 181202235809 -- 1103 ****
**** 181202235819 -- 1104 ****
**** 181202235829 -- 1105 ****
**** 181202235839 -- 1106 ****
**** 181202235849 -- 1107 ****
**** 181202235859 -- 1108 ****
**** 181202235909 -- 1109 ****
**** 181202235919 -- 1110 ****
**** 181202235929 -- 1111 ****
**** 181202235939 -- 1112 ****
**** 181202235949 -- 1113 ****
**** 181202235959 -- 1114 ****
**** 181203000009 -- 1115 ****
**** 181203000019 -- 1116 ****
**** 181203000029 -- 1117 ****
**** 181203000039 -- 1118 ****
**** 181203000049 -- 1119 ****
**** 181

...

**** 181203025219 -- 2148 ****
**** 181203025229 -- 2149 ****
**** 181203025239 -- 2150 ****
**** 181203025249 -- 2151 ****
**** 181203025259 -- 2152 ****
**** 181203025309 -- 2153 ****
**** 181203025319 -- 2154 ****
**** 181203025329 -- 2155 ****
**** 181203025339 -- 2156 ****
**** 181203025349 -- 2157 ****
**** 181203025359 -- 2158 ****
**** 181203025409 -- 2159 ****
**** 181203025419 -- 2160 ****
**** 181203025429 -- 2161 ****
**** 181203025439 -- 2162 ****
**** 181203025449 -- 2163 ****
**** 181203025459 -- 2164 ****
**** 181203025509 -- 2165 ****
**** 181203025519 -- 2166 ****
**** 181203025529 -- 2167 ****
**** 181203025539 -- 2168 ****
**** 181203025549 -- 2169 ****
**** 181203025559 -- 2170 ****
**** 181203025609 -- 2171 ****
**** 181203025619 -- 2172 ****
**** 181203025629 -- 2173 ****
**** 181203025639 -- 2174 ****
**** 181203025649 -- 2175 ****
**** 181203025659 -- 2176 ****
**** 181203025709 -- 2177 ****
**** 181203025719 -- 2178 ****
**** 181203025729 -- 2179 ****
**** 181

**** 181203033629 -- 2413 ****
**** 181203033639 -- 2414 ****
**** 181203033649 -- 2415 ****
**** 181203033659 -- 2416 ****
**** 181203033709 -- 2417 ****
**** 181203033719 -- 2418 ****
**** 181203033729 -- 2419 ****
**** 181203033739 -- 2420 ****
**** 181203033749 -- 2421 ****
**** 181203033759 -- 2422 ****
**** 181203033809 -- 2423 ****
**** 181203033819 -- 2424 ****
**** 181203033829 -- 2425 ****
**** 181203033839 -- 2426 ****
**** 181203033849 -- 2427 ****
**** 181203033859 -- 2428 ****
**** 181203033909 -- 2429 ****
**** 181203033919 -- 2430 ****
**** 181203033929 -- 2431 ****
**** 181203033939 -- 2432 ****
**** 181203033949 -- 2433 ****
**** 181203033959 -- 2434 ****
**** 181203034009 -- 2435 ****
**** 181203034019 -- 2436 ****
**** 181203034029 -- 2437 ****
**** 181203034039 -- 2438 ****
**** 181203034049 -- 2439 ****
**** 181203034059 -- 2440 ****
**** 181203034109 -- 2441 ****
**** 181203034119 -- 2442 ****
**** 181203034129 -- 2443 ****
**** 181203034139 -- 2444 ****
**** 181

**** 181203042039 -- 2678 ****
**** 181203042049 -- 2679 ****
**** 181203042059 -- 2680 ****
**** 181203042109 -- 2681 ****
**** 181203042119 -- 2682 ****
**** 181203042129 -- 2683 ****
**** 181203042139 -- 2684 ****
**** 181203042149 -- 2685 ****
**** 181203042159 -- 2686 ****
**** 181203042209 -- 2687 ****
**** 181203042219 -- 2688 ****
**** 181203042229 -- 2689 ****
**** 181203042239 -- 2690 ****
**** 181203042249 -- 2691 ****
**** 181203042259 -- 2692 ****
**** 181203042309 -- 2693 ****
**** 181203042319 -- 2694 ****
**** 181203042329 -- 2695 ****
**** 181203042339 -- 2696 ****
**** 181203042349 -- 2697 ****
**** 181203042359 -- 2698 ****
**** 181203042409 -- 2699 ****
**** 181203042419 -- 2700 ****
**** 181203042429 -- 2701 ****
**** 181203042439 -- 2702 ****
**** 181203042449 -- 2703 ****
**** 181203042459 -- 2704 ****
**** 181203042509 -- 2705 ****
**** 181203042519 -- 2706 ****
**** 181203042529 -- 2707 ****
**** 181203042539 -- 2708 ****
**** 181203042549 -- 2709 ****
**** 181

**** 181203050449 -- 2943 ****
**** 181203050459 -- 2944 ****
**** 181203050509 -- 2945 ****
**** 181203050519 -- 2946 ****
**** 181203050529 -- 2947 ****
**** 181203050539 -- 2948 ****
**** 181203050549 -- 2949 ****
**** 181203050559 -- 2950 ****
**** 181203050609 -- 2951 ****
**** 181203050619 -- 2952 ****
**** 181203050629 -- 2953 ****
**** 181203050639 -- 2954 ****
**** 181203050649 -- 2955 ****
**** 181203050659 -- 2956 ****
**** 181203050709 -- 2957 ****
**** 181203050719 -- 2958 ****
**** 181203050729 -- 2959 ****
**** 181203050739 -- 2960 ****
**** 181203050749 -- 2961 ****
**** 181203050759 -- 2962 ****
**** 181203050809 -- 2963 ****
**** 181203050819 -- 2964 ****
**** 181203050829 -- 2965 ****
**** 181203050839 -- 2966 ****
**** 181203050849 -- 2967 ****
**** 181203050859 -- 2968 ****
**** 181203050909 -- 2969 ****
**** 181203050919 -- 2970 ****
**** 181203050929 -- 2971 ****
**** 181203050939 -- 2972 ****
**** 181203050949 -- 2973 ****
**** 181203050959 -- 2974 ****
**** 181

**** 181203054859 -- 3208 ****
**** 181203054909 -- 3209 ****
**** 181203054919 -- 3210 ****
**** 181203054929 -- 3211 ****
**** 181203054939 -- 3212 ****
**** 181203054949 -- 3213 ****
**** 181203054959 -- 3214 ****
**** 181203055009 -- 3215 ****
**** 181203055019 -- 3216 ****
**** 181203055029 -- 3217 ****
**** 181203055039 -- 3218 ****
**** 181203055049 -- 3219 ****
**** 181203055059 -- 3220 ****
**** 181203055109 -- 3221 ****
**** 181203055119 -- 3222 ****
**** 181203055129 -- 3223 ****
**** 181203055139 -- 3224 ****
**** 181203055149 -- 3225 ****
**** 181203055159 -- 3226 ****
**** 181203055209 -- 3227 ****
**** 181203055219 -- 3228 ****
**** 181203055229 -- 3229 ****
**** 181203055239 -- 3230 ****
**** 181203055249 -- 3231 ****
**** 181203055259 -- 3232 ****
**** 181203055309 -- 3233 ****
**** 181203055319 -- 3234 ****
**** 181203055329 -- 3235 ****
**** 181203055339 -- 3236 ****
**** 181203055349 -- 3237 ****
**** 181203055359 -- 3238 ****
**** 181203055409 -- 3239 ****
**** 181

In [ ]:
sys.exit()

## Read the data and compare to the command line method to see how it worked

In [ ]:
# %load_ext autoreload
# %autoreload 2
# import copy
# import time
# import sys  

# # import pickle 
# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn
# ### Identify which arcs you want to run:
# arcs_files = [ 
#             '2018.336',  # 1
#              ]

# Obj_Geodyn = {}
# #------ A dictionary containing the run parameters ------  
# run_params = {} 
# run_params['arc']              =   arcs_files
# run_params['satellite']        =  'icesat2'  
# run_params['den_model']        =  'tiegcm_oc'  
# run_params['SpecialRun_name']  =  'PCE_kamodo_interface'  
# run_params['verbose']          =  False
# run_params['geodyn_StepSize']  =  float(10.)

# for imodel,val_model in enumerate( ['tiegcm_oc', ]):
#     read_params = copy.deepcopy(run_params)
#     read_params['den_model']  =  val_model  
#     read_params['action']  =  'read'  
#     read_params['request_data']      = ['AdjustedParams',
#                                         'Trajectory_orbfil',
#                                         'Density',
#                                         'Residuals_obs',
#                                         'Residuals_summary',
#                                         ]

#     ### Load the data into an object
#     Obj_Geodyn[val_model] = Pygeodyn(read_params)
#     Obj_Geodyn[val_model].getData()


In [ ]:
Obj_Geodyn['tiegcm_oc'].__dict__['AdjustedParams']['2018.336'][4][1807001]['0CD']


## Plots

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px


config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

## Plot- RMS Summary

In [ ]:
%load_ext autoreload
%autoreload 2

from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_residual_meas_summary
from PYGEODYNAnalysis_icesat2PCEtrajectory import rms_summary_table


Obj_list = [Obj_Geodyn['tiegcm_oc']]

rms_summary_table(Obj_list)


fig = make_subplots(rows=2, cols=1, 
     subplot_titles=(["Mean Residuals per Arc", 'RMS of Fit per Arc']),
     vertical_spacing = 0.1)
fig = plot_residual_meas_summary(fig, Obj_Geodyn['tiegcm_oc']   , 0)

fig.show(config=config)

### CD timeseries

In [ ]:
%load_ext autoreload
%autoreload 2
from PYGEODYNAnalysis_icesat2PCEtrajectory import plot_cd_and_percdiff_from_apriori


fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(["Timeseries of Cd", "Ratio of Adjusted Cd to a priori (Cd=2.2)"]),
    vertical_spacing = 0.08,
    )
fig = plot_cd_and_percdiff_from_apriori(fig,  Obj_Geodyn['tiegcm_oc']     , 0)


fig.show(config=config)

### Plot - NTW Residuals

In [ ]:
%load_ext autoreload
%autoreload 2
from PYGEODYNAnalysis_icesat2PCEtrajectory import NTW_CDratio_IntrackResids


fig = make_subplots(rows=2, cols=1,
            shared_xaxes=True,
            subplot_titles=(['Adjusted Cd ratio to a priori (2.2)', 'In-Track Component Residuals (PCE-ORBFIL)']),
            vertical_spacing = 0.1,
            specs=[ [{"secondary_y": False }],
                    [{"secondary_y": False }]],)


fig = NTW_CDratio_IntrackResids(fig,  Obj_Geodyn['tiegcm_oc']  , 0)


fig.show(config=config)